# 

In [ ]:
import os,sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D #gehört zu matplotlib.pyplot
import seaborn as sns
#import mpl_toolkits.mplot3d

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
print(tf.__version__)

## Helper Funktions

In [ ]:
def plot_loss(history):
    plt.figure()
    plt.plot(history.history['loss'], label='loss')
    
    if 'val_loss' in history.history:
        plt.plot(history.history['val_loss'], label='val_loss')
    
    plt.xlabel('Epoch')
    plt.ylabel('Mean Squared Error')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
notebooks_folder = os.getcwd()
projekt_folder = os.path.dirname(notebooks_folder)
source_data=os.path.join(projekt_folder,"data","harvest.csv")

df_source=pd.read_csv(source_data)
df_source.head(5)
#print(df_source.head(5))


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection="3d")

ax.scatter(
    df_source["Dünger"],
    df_source["Niederschlag"],
    df_source["Ertrag"]
)

ax.set_xlabel("Dünger")
ax.set_ylabel("Niederschlag")
ax.set_zlabel("Ertrag")
ax.set_title("3D-Plot der Ertragsdaten")

ax.view_init(elev=25, azim=135)


plt.show()



In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.scatterplot(
    data=df_source,
    x="Dünger",
    y="Ertrag",
    ax=axes[0]
)
axes[0].set_title("Dünger vs. Ertrag")

sns.scatterplot(
    data=df_source,
    x="Niederschlag",
    y="Ertrag",
    ax=axes[1]
)
axes[1].set_title("Niederschlag vs. Ertrag")

plt.tight_layout()
plt.show()


# Clean the Data

In [ ]:
df_source.isna().sum()
#dataset = dataset.dropna() #nicht nötig da kein Nan

seperate Data sets for learning

In [ ]:
train_dataset = df_source.sample(frac=0.8, random_state=0)
test_dataset = df_source.drop(train_dataset.index)



Set the Labels

In [ ]:
train_features = train_dataset[["Dünger", "Niederschlag"]]
test_features  = test_dataset[["Dünger", "Niederschlag"]]

train_labels = train_dataset["Ertrag"]
test_labels  = test_dataset["Ertrag"]

# Normalization

In [ ]:
train_dataset.describe().transpose()[['mean', 'std']]

In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))
print("Features Normalized!")
print(normalizer.mean.numpy())
print()

first = np.array(train_features[:1])
with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

# Regression with deep neural network

## Build the model Function!

In [ ]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(4, activation='relu'),
      layers.Dense(4, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_squared_error',
                optimizer='adam')#tf.keras.optimizers.Adam(0.001))
  return model

In [ ]:
dnn_ertrags_model = build_and_compile_model(normalizer)

In [ ]:
# No Model ist Build yet!
dnn_ertrags_model.summary()

Traing des Models

In [ ]:
history = dnn_ertrags_model.fit(
    train_features,
    train_labels,
    epochs=100,
    batch_size=50,
    validation_split=0.2)#,
    #verbose=0)

In [ ]:
dnn_ertrags_model.summary()

In [ ]:
plot_loss(history)